In [1]:
import random
import numpy as np
import sys
from random import randint
from data_load import *


In [2]:
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")

In [3]:

def train(A,B,insts,size,lr,max_it =10):
    for it in xrange(1,max_it+1):
        error = 0.

        for i,inst in enumerate(insts):
            error+=gradient(A,B,inst,size,lr=lr)
        
            if i % 1000 ==0:
                sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
                sys.stdout.flush()
        sys.stdout.write("\n")
def gradient(A,B,inst,size,lr=0.01):
    #TODO

    random.seed(1)
    dA = np.zeros(size)
    dB = np.zeros([len(inst.labels),size])
    x = np.sum(A[inst.features],axis=0)
    error = 0.
    neg_num = len(inst.negative_labels)
    for l in inst.sparse_labels:
        s1= np.vdot(x,B[l])
        N=1
        n_sample  = -1
        for k in xrange(neg_num):
            nl = inst.negative_labels[randint(0,neg_num-1)]
            s2 = np.vdot(x,B[nl])
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:
            L = rank(len(inst.negative_labels)/N)            
            error += (1+s2-s1)*L

            dA += L*(B[l]-B[n_sample])
            dB[l] += L*x
            dB[nl] -= L*x
    for f in inst.features:
        A[f] += lr*dA
        norm = np.linalg.norm(A[f])
        if norm >1:
            A[f] /= norm
    for i in xrange(len(B)):
        B[i] += lr*dB[i]
        norm =  np.linalg.norm(B[i])
        if norm >1:
            B[i] /=norm
    return error
def rank(k):
    loss = 0.
    for i in xrange(1,k+1):
        loss += 1./i
    return loss
def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")
    

In [5]:
size=50
np.random.seed(1)

A= np.random.rand(len(a.feature2id),size).astype(np.float32)
B= np.random.rand(len(a.label2id),size).astype(np.float32)
train(A,B,a.data,50,lr=0.01,max_it=20)
save_to_text(A,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_A.txt')
save_to_text(B,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_B.txt')

Iteration 1 trained 26% Loss:288207.06

KeyboardInterrupt: 

Iteration 1 trained 0% Loss:170.92
Iteration 2 trained 0% Loss:1.64
